# Drawing Monet --- Generate a Monet-style painting.

How do I generate a fake Monet?
This competition suggests GAN, but it is not the only way for forgers to do this.
Here we will use a somewhat older method - the ' Neural Algorithm of Artistic Style' to create a forgery.

This method takes the features of the image from the VGG and propagates them back to the input image itself (rather than the parameters in the neural network). This (of course) changes the input image.
In the end, the input image is a hybrid of the style image and the original image.

For more information, please refer to the following original paper
[https://arxiv.org/abs/1508.06576](https://arxiv.org/abs/1508.06576)

In [ ]:
import torch
from torch import nn, utils, optim
import torchvision as tv
import os
import io
import zipfile
from tqdm.notebook import tqdm
from PIL import Image
import imageio
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from matplotlib import pyplot as plt

In [ ]:
to_tensor = tv.transforms.Compose([
                tv.transforms.Resize((256,256)),
                tv.transforms.ToTensor(),
                tv.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[1, 1, 1]),
            ])

unload = tv.transforms.Compose([
                tv.transforms.Normalize(mean=[-0.485,-0.456,-0.406],
                                    std=[1,1,1]),                
                tv.transforms.Lambda(lambda x: x.clamp(0,1))
            ])
to_image = tv.transforms.ToPILImage()

In [ ]:
class MyDataset:
    def __init__(self, style=False, target='../input/gan-getting-started/photo_jpg'):
        self.style = style
        self.target = target
        self.x = os.listdir(self.target)
        if style:
            self.y = os.listdir('../input/gan-getting-started/monet_jpg')

    def __len__(self):
        return min(7000, len(self.x))

    def __getitem__(self, pos):
        input_img = self.target + '/' + self.x[pos]
        input_img = Image.open(input_img)
        input_img = to_tensor(input_img)
        if not self.style:
            return input_img
        style_img = '../input/gan-getting-started/monet_jpg/' + self.y[style_choice[pos]]
        style_img = Image.open(style_img)
        style_img = to_tensor(style_img)
        return input_img, style_img

# Obtain image features from VGG19

This step has no relation with the Neural Algorithm of Artistic Style yet. We will try to use VGG for image vectorization.

In [ ]:
VGG = tv.models.vgg19(pretrained=False)
VGG.load_state_dict(torch.load('../input/vgg19dcbb9e9dpth/vgg19-dcbb9e9d.pth'))
VGG.cuda()
VGG.classifier = VGG.classifier[:4]

# Find the Monet that is simillar to the original photos.

The step select the style image.

It is not necessary if you want to select the style image randomly, but here we select the Monet that most simillar to original photo as the style image in order to make a better forgery.

In [ ]:
VGG.eval()
photo_vecs = []
dataset = MyDataset(False)
data_loader = utils.data.DataLoader(
    dataset, batch_size=64, shuffle=False, num_workers=4)
for x in tqdm(data_loader):
    f = VGG(x.cuda())
    f.detach().cpu().numpy()
    photo_vecs.extend(f.tolist())
monet_vecs = []
dataset = MyDataset(False, target='../input/gan-getting-started/monet_jpg')
data_loader = utils.data.DataLoader(
    dataset, batch_size=64, shuffle=False, num_workers=4)
for x in tqdm(data_loader):
    f = VGG(x.cuda())
    f.detach().cpu().numpy()
    monet_vecs.extend(f.tolist())
style_choice = cosine_similarity(photo_vecs, monet_vecs)
style_choice = np.argmax(style_choice, axis=1)

# VGG features

Unique about Leon's work is that what part of the VGG features includes the style or shape.

In [ ]:
def get_features(module, x, y):
    features.append(y)
    
def gram_matrix(x):
    b, c, h, w = x.size()
    F = x.view(b,c,h*w)
    G = torch.bmm(F, F.transpose(1,2))/(h*w)
    return G

In [ ]:
VGG = VGG.features
VGG.eval()

features = []

for i, layer in enumerate(VGG):
    
    if i in [0,5,10,19,21,28]:
        VGG[i].register_forward_hook(get_features)
    
    elif isinstance(layer, nn.MaxPool2d):
        VGG[i] = nn.AvgPool2d(kernel_size=2)

# Make Fake Monet.

Mix and back-propagate shape features and style features.

In [ ]:
def make_monet(input_imgs, style_imgs):
    global features
    
    for p in VGG.parameters():
        p.requires_grad = False

    features = []
    VGG(input_imgs)
    c_target = features[4].detach()

    features = []
    VGG(style_imgs)
    f_targets = features[:4]+features[5:]
    gram_targets = [gram_matrix(i).detach() for i in f_targets]
    
    alpha = 1
    beta = 1e3
    iterations = 5
    image = input_imgs.clone()
    optimizer = optim.LBFGS([image.requires_grad_()], lr=1)    
    mse_loss = nn.MSELoss(reduction='mean')
    l_c = []
    l_s = []
    counter = 0
    
    for itr in range(iterations):

        features = []
        def closure():
            optimizer.zero_grad()
            VGG(image)
            t_features = features[-6:]
            content = t_features[4]
            style_features = t_features[:4]+t_features[5:]
            t_features = []
            gram_styles = [gram_matrix(i) for i in style_features]
            c_loss = alpha * mse_loss(content, c_target)
            s_loss = 0

            for i in range(5):
                n_c = gram_styles[i].shape[0]
                s_loss += beta * mse_loss(gram_styles[i],gram_targets[i])/(n_c**2)

            total_loss = c_loss+s_loss

            l_c.append(c_loss)
            l_s.append(s_loss)

            total_loss.backward()
            return total_loss

        optimizer.step(closure)
    
    result = []
    for i in range(len(X)):
        temp = unload(image[i].cpu().detach())
        temp = to_image(temp)
        temp = np.array(temp)
        result.append(temp)
    return result

# Samples

Visualize 4 sample fake Monets.

In [ ]:
dataset = MyDataset(True)
data_loader = utils.data.DataLoader(
    dataset, batch_size=4, shuffle=False, num_workers=4)

In [ ]:
for X, y in data_loader:
    i = make_monet(X.cuda(), y.cuda())
    break
plt.imshow(i[0])

In [ ]:
plt.imshow(i[1])

In [ ]:
plt.imshow(i[2])

In [ ]:
plt.imshow(i[3])

# Make Submission

This task takes a long time - because it is generated by back propagation.

Normaly, Pre-trained StyleGAN are better because they are faster to generate. However, this method also has the advantage of not requiring any training.

Probably, the number of 7000 images specified by the Kaggle team is considering that such a method will not work within the time limit of the kernel.

However, we clear this by limiting the number of back propagations to 5. If we increase the number more, the quality will improve, but it will be trapped by the competition time limit.

In [ ]:
dataset = MyDataset(True)
data_loader = utils.data.DataLoader(
    dataset, batch_size=8, shuffle=False, num_workers=4)

In [ ]:
write_count = 0
with zipfile.ZipFile("images.zip", "w", zipfile.ZIP_DEFLATED, False) as zip_file:
    for X, y in tqdm(data_loader):
        imgs = make_monet(X.cuda(), y.cuda())
        for img in imgs:
            with io.BytesIO() as data:
                Image.fromarray(img).save(data, format='PNG')
                img_bytes = data.getvalue()
            zip_file.writestr(f"{write_count}.png", img_bytes)
            write_count += 1